In [60]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime
import json

#from lxml import html

In [89]:
listURL = []
ListCourseName = []

for i in range(3):
    GradeSourceURL = 'http://www.gradesource.com/findcourse.asp?sb=C&name=cse&st=' + str(10 * i)
    getGradeSource = requests.get(GradeSourceURL)
    gradeSoup = BeautifulSoup(getGradeSource.content, 'html.parser')
    
    listLink = gradeSoup.find_all('tr', {'bgcolor': '#FFFFFF'})
    listName = gradeSoup.find_all('font', {'size': '3'})
    tempLinkList = gradeSoup.find_all('a')
    counter = 0
    
    
    for i in range(len(tempLinkList)):
        if tempLinkList[i]['href'][:26] == 'http://www.gradesource.com':
            searchURL = requests.get(tempLinkList[i]['href'])
            findURLSoup = BeautifulSoup(searchURL.content, 'html.parser')
            
            if len(findURLSoup.find_all('a', {'href': 'coursestand.html'})) != 0:
                listURL.append(tempLinkList[i]['href'][:-10] + findURLSoup.find_all('a', {'href': 'coursestand.html'})[0]['href'])
                ListCourseName.append(listName[counter].find('b').text)
            else:
                print("Doesn't exist")
            counter += 1
                     


Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist


In [90]:
for  i in range(len(ListCourseName)):
    print(ListCourseName[i], listURL[i])

CSE 100 - Advanced Data Structures http://www.gradesource.com/reports/5899/29957/coursestand.html
CSE 11 - Introduction to Programming in Java http://www.gradesource.com/reports/5899/29958/coursestand.html
CSE 110 - Software Engineering http://www.gradesource.com/reports/7/29889/coursestand.html
CSE 12 - Basic Data Structures & Object-Oriented Design http://www.gradesource.com/reports/7/29887/coursestand.html
CSE 120 - Principle of Computer Operating Systems http://www.gradesource.com/reports/1917/29997/coursestand.html
CSE 127 (Gantman) - Introduction to Computer Security http://www.gradesource.com/reports/4955/30018/coursestand.html
CSE 127 (Savage) - Introduction to Computer Security http://www.gradesource.com/reports/4955/30017/coursestand.html
CSE 132B - Database System Applications http://www.gradesource.com/reports/1918/29968/coursestand.html
CSE 150 - Introduction to AI: Probabilistic Reasoning and Decision Making http://www.gradesource.com/reports/2636/29970/coursestand.html
C

In [2]:
#Here is a list of URLS, please update with current
ListOfURL = ['http://www.gradesource.com/reports/288/29963/coursestand.html', 'http://www.gradesource.com/reports/7/29889/cs126276.html', 'http://www.gradesource.com/reports/7/29887/coursestand.html', 'http://www.gradesource.com/reports/430/29962/coursestand.html']
ListOfCourse = ['CSE30_Sp18', 'CSE110_Sp18', 'CSE12_Sp18', 'CSE7_Sp18']

In [91]:
def getGradeSourceGrades(URL):
    arrTotal = []

    quote_page = URL
    #URL of gradesource website that needs to be scraped
    page = requests.get(quote_page)
    #making a request
    soup = BeautifulSoup(page.content, 'html.parser')
    #cleaning the html
    
    table = soup.find_all('tr')
    #retrieving all the table
    arr = []

    for i in range(len(table)):
        arr.append(table[i].find_all('td'))

    arrHolder = {}
    for i in range(len(arr)):
        if i >= 8 and i < len(arr):     
            if arr[i][0].find("b") != None:

                arrHolder[arr[i][0].find("b").text] = {}
                
                if arr[i][len(arr[i])-1].find("b") != None:
               
                    arrHolder[arr[i][0].find("b").text]["Grade"] = arr[i][len(arr[i])-1].find("b").text  # getting the course id
              
                elif arr[i][len(arr[i])-1].find("font") != None:
                
                    arrHolder[arr[i][0].find("b").text]["Grade"] = arr[i][len(arr[i])-1].find("font").text # getting the rank
           
                arrHolder[arr[i][0].find("b").text]["Rank"] = arr[i][len(arr[i])-2].find("font").text + "/" + str(len(arr)-4)
            elif arr[i][0].find("font") != None:

                arrHolder[arr[i][0].find("font").text] = {}
                
                if arr[i][len(arr[i])-1].find("b") != None:

                    arrHolder[arr[i][0].find("font").text]["Grade"] = arr[i][len(arr[i])-1].find("b").text # getting the course id

                elif arr[i][len(arr[i])-1].find("font") != None:

                    arrHolder[arr[i][0].find("font").text]["Grade"] = arr[i][len(arr[i])-1].find("font").text # getting the rank
                
                arrHolder[arr[i][0].find("font")]["Rank"] = str(arr[i][len(arr[i])-2].find("font").text) + "/" + str(len(arr)-4)
    #arrTotal.append(arrHolder)
    return arrHolder

In [96]:
#this code is used for formatting into a json 
def runGradeSource():
    

    listURL = []
    ListCourseName = []

    # This code gets the most recent gradesource
    for i in range(3):
        GradeSourceURL = 'http://www.gradesource.com/findcourse.asp?sb=C&name=cse&st=' + str(10 * i)
        getGradeSource = requests.get(GradeSourceURL)
        gradeSoup = BeautifulSoup(getGradeSource.content, 'html.parser')
    
        listLink = gradeSoup.find_all('tr', {'bgcolor': '#FFFFFF'})
        listName = gradeSoup.find_all('font', {'size': '3'})
        tempLinkList = gradeSoup.find_all('a')
        counter = 0
    
    
        for i in range(len(tempLinkList)):
            if tempLinkList[i]['href'][:26] == 'http://www.gradesource.com':
                searchURL = requests.get(tempLinkList[i]['href'])
                findURLSoup = BeautifulSoup(searchURL.content, 'html.parser')
                    
                if len(findURLSoup.find_all('a', {'href': 'coursestand.html'})) != 0:
                    listURL.append(tempLinkList[i]['href'][:-10] + findURLSoup.find_all('a', {'href': 'coursestand.html'})[0]['href'])
                    ListCourseName.append(listName[counter].find('b').text)
                else:
                    print("Doesn't exist")
                counter += 1
        
       ##### 
    for i in range(len(listURL)):
        from firebase import firebase
        CSECourses = getGradeSourceGrades(listURL[i])

        courseList = json.dumps(CSECourses, indent=4)
        courseList = courseList.replace("/", "／")
        courseList = courseList.replace(".", "．")

        jsonDict = json.loads(courseList)

        #print(courseList)
        jsonDict = json.loads(courseList)
        firebase = firebase.FirebaseApplication('https://csehub-420.firebaseio.com/')

        #This code is intended to send the dictionary to firebase
        result = firebase.patch('/GradeSource', { ListCourseName[i]: jsonDict})

In [97]:
runGradeSource()

Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist
Doesn't exist


In [56]:
scheduler = BackgroundScheduler()
scheduler.add_job(runGradeSource, 'interval', minutes=3, next_run_time=datetime.now())
# for yearly scraping
# scheduler.add_job(scrapeCapes, 'interval', years=1, next_run_time=datetime.now())
scheduler.start()

Job "runGradeSource (trigger: interval[0:03:00], next run at: 2018-05-29 22:47:12 PDT)" raised an exception
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/apscheduler/executors/base.py", line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "<ipython-input-55-93703a430752>", line 3, in runGradeSource
    for i in range(len(ListURL)):
NameError: name 'ListURL' is not defined
